In [2]:
# this block of code imports graphical libraries for plotting graphs with high resolution
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 120

In [3]:
!pip install -q usearch


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
# Libraries of functions need to be imported
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
from scipy.spatial.distance import cdist
from usearch.index import search, MetricKind, Matches, BatchMatches
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from scipy.spatial import Delaunay
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import mean_squared_error as mse
from scipy import linalg
from sklearn.base import BaseEstimator, RegressorMixin
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from scipy.spatial.distance import cdist
from scipy.interpolate import interp1d
from sklearn.utils.validation import check_is_fitted
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.exceptions import NotFittedError
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted

In [5]:
data = pd.read_csv('https://github.com/dvasiliu/AAML/blob/main/Data%20Sets/mtcars.csv?raw=True')
data

,model,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
0,Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
1,Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
2,Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
3,Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
4,Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2
5,Valiant,18.1,6,225.0,105,2.76,3.460,20.22,1,0,3,1
6,Duster 360,14.3,8,360.0,245,3.21,3.570,15.84,0,0,3,4
7,Merc 240D,24.4,4,146.7,62,3.69,3.190,20.00,1,0,4,2
8,Merc 230,22.8,4,140.8,95,3.92,3.150,22.90,1,0,4,2
9,Merc 280,19.2,6,167.6,123,3.92,3.440,18.30,1,0,4,4


In [6]:
# We will be having multiple features - cyl, disp, hp, drat, wt, qsec, vs, am, gear, carb
X = data.drop(columns=['model', 'mpg']).values
y = data['mpg'].values

In [7]:
# if we compute distances we need to put the attributes on the same scale
scaler = StandardScaler()
xscaled = scaler.fit_transform(x)

NameError: name 'x' is not defined

In [ ]:
many_in_many: BatchMatches = search(xscaled, xscaled,392 , MetricKind.L2sq, exact=True)

In [ ]:
one_in_many: Matches = search(xscaled, xscaled[0], len(xscaled), MetricKind.L2sq, exact=True)

In [ ]:
dist = one_in_many.to_list()

In [ ]:
distances = cdist(xscaled,xscaled,metric='Euclidean')

In [ ]:
distances.shape

(392, 392)

### Kernels for the Weights

In [ ]:
# Gaussian Kernel
def Gaussian(x):
  return np.where(np.abs(x)>4,0,1/(np.sqrt(2*np.pi))*np.exp(-1/2*x**2))

In [ ]:
# this is the correct vectorized version
def Tricubic(x):
  return np.where(np.abs(x)>1,0,(1-np.abs(x)**3)**3)

In [ ]:
# Epanechnikov Kernel
def Epanechnikov(x):
  return np.where(np.abs(x)>1,0,3/4*(1-np.abs(x)**2))

In [ ]:
# Quartic Kernel
def Quartic(x):
  return np.where(np.abs(x)>1,0,15/16*(1-np.abs(x)**2)**2)

In [ ]:
def kernel_function(d_ij,kern, tau):
    return kern(d_ij/(2*tau))

In [ ]:
# we want the weights
kern = Gaussian

In [ ]:
weights = kernel_function(distances,kern,tau=0.05)

In [ ]:
model = Ridge(alpha=0.001,max_iter=5000)

In [ ]:
model.fit(np.diag(weights[:,0])@xscaled, np.diag(weights[:,0])@y)

Ridge(alpha=0.001, max_iter=5000)

In [ ]:
model.predict(xscaled[0].reshape(1,-1))

array([17.06248486])

In [ ]:
y[0]

18.0

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
# Add Cauchy Kernel
def Cauchy(x):
    return 1 / (1 + x**2)

def Tricubic(x):
    return np.where(np.abs(x) > 1, 0, (1 - np.abs(x)**3)**3)


In [ ]:
def kernel_function(distances, kernel, tau):
    return kernel(distances / (2 * tau))

def calculate_distances(X):
    return cdist(X, X, metric='euclidean')


In [ ]:
class Lowess(BaseEstimator, RegressorMixin):
    def __init__(self, kernel=Cauchy, tau=0.1, regularization=0.001):
        self.kernel = kernel
        self.tau = tau
        self.regularization = regularization
        self.fitted_ = False  

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y
        self.distances = calculate_distances(self.X_train)
        self.weights = kernel_function(self.distances, self.kernel, self.tau)
        self.fitted_ = True  

    def predict(self, X_new):
        if not self.fitted_:
            raise NotFittedError(f"This Lowess instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.")

        predictions = []

        for i in range(len(X_new)):
            w = self.weights[:, i]
            model = Ridge(alpha=self.regularization)
            model.fit(np.diag(w) @ self.X_train, np.diag(w) @ self.y_train)
            prediction = model.predict(X_new[i].reshape(1, -1))
            predictions.append(prediction)

        return np.array(predictions)

    def get_params(self, deep=True):
        return {"kernel": self.kernel, "tau": self.tau, "regularization": self.regularization}

    def set_params(self, **params):
        for key, value in params.items():
            setattr(self, key, value)
        return self


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

model = Lowess(kernel=Cauchy, tau=0.15)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print(y_pred[:5])
print(y_test[:5])


[[ 9.5495056 ]
 [31.74452208]
 [10.16949669]
 [ 5.49252384]
 [31.89069237]]
[26.       21.6      36.099998 26.       27.      ]


In [ ]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")


Mean Squared Error: 2493.717088612443


In [ ]:
param_grid = {'tau': np.linspace(0.01, 0.5, 10)}
grid_search = GridSearchCV(Lowess(), param_grid, scoring='neg_mean_squared_error', cv=5)

grid_search.fit(X_train, y_train)
best_tau = grid_search.best_params_['tau']

print(f"Best tau: {best_tau}")

best_model = Lowess(kernel=Cauchy, tau=best_tau)
best_model.fit(X_train, y_train)

best_y_pred = best_model.predict(X_test)
best_mse = mean_squared_error(y_test, best_y_pred)
print(f"Optimized Mean Squared Error: {best_mse}")


Best tau: 0.5
Optimized Mean Squared Error: 351.41695387566307


In [ ]:
pip install mercury

In [10]:
import mercury as mr
 
# configure App properties
app = mr.App(title="Static notebook", description="Display static notebook", static_notebook=True)

mercury.App